# 🔄 Modification des Sources de Candidatures

Ce notebook permet de modifier les sources des candidatures dans l'API.

## 🎯 Objectif

Ce notebook permet de :
- Récupérer toutes les candidatures depuis l'API
- Analyser les différentes sources existantes
- Modifier les sources des candidatures selon vos critères
- Sauvegarder les données avant et après modification

---

## 📋 Guide d'utilisation sur Google Colab

### Étape 1 : Ouvrir le notebook sur Google Colab

1. Allez sur [Google Colab](https://colab.research.google.com/)
2. Téléversez ce fichier `.ipynb` dans Colab
3. Le notebook s'ouvrira automatiquement

### Étape 2 : Configurer les paramètres

1. Exécutez la cellule de **configuration** (Étape 3)
2. Vérifiez que l'API URL et la clé API sont correctes
3. Définissez quelle source vous voulez changer

### Étape 3 : Exécuter toutes les cellules

**Méthode rapide :**
- Menu **"Runtime"** → **"Run all"** (Exécuter tout)

**Méthode manuelle :**
- Exécutez chaque cellule dans l'ordre (Shift + Entrée)

### Étape 4 : Vérifier les résultats

Les fichiers CSV seront générés dans le panneau de fichiers à gauche.

---

## ⚠️ Attention

**Ce notebook modifie les données dans l'API !**

- Assurez-vous d'avoir sauvegardé les données avant de modifier
- Vérifiez bien les paramètres de configuration
- Testez d'abord sur un petit échantillon si possible

## 💾 Système de Backup et Restauration

**✅ Backup automatique :**
- Un backup complet est créé automatiquement avant toute modification
- Un fichier de mapping (ID → source originale) est généré pour faciliter la restauration
- Tous les fichiers sont sauvegardés avec un timestamp pour éviter les écrasements

**🔄 Restauration possible :**
- Si le frontend ne peut pas afficher la nouvelle source "cibli", vous pouvez restaurer
- Une section complète de restauration est disponible à la fin du notebook
- La restauration remet toutes les sources à leur valeur d'origine exacte

**📁 Fichiers de backup importants :**
- `all_applications_backup_YYYYMMDD_HHMMSS.csv` - Backup complet
- `restore_mapping_YYYYMMDD_HHMMSS.csv` - Mapping pour restauration (⚠️ GARDEZ-LE !)

---

**Prêt à commencer ? Configurez d'abord les paramètres ci-dessous ! 👇**


## 📦 Étape 1 : Installation des bibliothèques nécessaires

Cette cellule installe tous les outils nécessaires.

**⚠️ Important :** Exécutez cette cellule en premier !

**Temps estimé :** 10-30 secondes


In [1]:
%pip install pandas requests


Note: you may need to restart the kernel to use updated packages.


## 📚 Étape 2 : Importation des bibliothèques

Cette cellule charge les outils Python nécessaires.

**Aucune action requise** - Exécutez simplement la cellule.


In [2]:
import requests
import pandas as pd
import json
import warnings
import os
from datetime import datetime

warnings.filterwarnings('ignore')


## ⚙️ Étape 3 : Configuration

**⚠️ IMPORTANT :** Modifiez ces paramètres selon vos besoins !

- `API_URL` : L'URL de base de l'API
- `API_KEY` : Votre clé API (x-api-key)
- `SOURCE_TO_KEEP` : La source à conserver (ne sera pas modifiée)
- `NEW_SOURCE` : La nouvelle source à attribuer aux autres candidatures

**Exemple :** Si vous voulez garder "cabine cibli job" et changer toutes les autres en "cibli" :
- `SOURCE_TO_KEEP = "cabine cibli job"`
- `NEW_SOURCE = "cibli"`


In [6]:
# Configuration de l'API
API_URL = "https://api.smart-process-rh.com/v1"
API_KEY = "9TTaz70w8biMjvJ9Q5eIHZwVlQRNmjqAqiNzyGjfeI1S4nubpkSAL1h87FoNrlMv"

# Configuration des sources
SOURCE_TO_KEEP = "cabine cibli job"  # Cette source ne sera PAS modifiée
NEW_SOURCE = "cibli"  # Toutes les autres sources seront changées en ceci

# Endpoint pour récupérer toutes les candidatures
APPLICATIONS_ENDPOINT = f"{API_URL}/applications/all"

print("✅ Configuration chargée :")
print(f"   API URL: {API_URL}")
print(f"   Source à conserver: '{SOURCE_TO_KEEP}'")
print(f"   Nouvelle source: '{NEW_SOURCE}'")


✅ Configuration chargée :
   API URL: https://api.smart-process-rh.com/v1
   Source à conserver: 'cabine cibli job'
   Nouvelle source: 'cibli'


## 📥 Étape 4 : Téléchargement de toutes les candidatures

Cette cellule :
- Récupère toutes les candidatures depuis l'API
- Sauvegarde les données brutes dans un fichier CSV
- Affiche un résumé des données récupérées

**Temps estimé :** 10-30 secondes selon le nombre de candidatures

**Résultat attendu :** Un message `<Response [200]>` si tout fonctionne correctement.


In [7]:
# Récupération de toutes les candidatures
print("📥 Récupération des candidatures depuis l'API...")

response = requests.get(
    APPLICATIONS_ENDPOINT,
    headers={"x-api-key": API_KEY}
)

print(f"Réponse API: {response}")

if response.status_code == 200:
    applications_data = response.json()
    applications_df = pd.DataFrame(applications_data)
    
    # Créer le dossier si nécessaire
    if not os.path.exists('applications'):
        os.makedirs('applications')
    
    # Sauvegarder avec timestamp pour éviter d'écraser les anciennes données
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    backup_filename = f'applications/all_applications_backup_{timestamp}.csv'
    applications_df.to_csv(backup_filename, index=False)
    
    # Créer un mapping de restauration (ID -> source originale) pour faciliter la restauration
    if 'id' in applications_df.columns and 'source' in applications_df.columns:
        restore_mapping = applications_df[['id', 'source']].copy()
        restore_mapping.columns = ['application_id', 'original_source']
        restore_mapping_filename = f'applications/restore_mapping_{timestamp}.csv'
        restore_mapping.to_csv(restore_mapping_filename, index=False)
        print(f"\n✅ Mapping de restauration sauvegardé dans: {restore_mapping_filename}")
        print(f"   Ce fichier permet de restaurer toutes les sources originales facilement")
    
    print(f"\n✅ {len(applications_df)} candidatures récupérées")
    print(f"✅ Backup complet sauvegardé dans: {backup_filename}")
    print(f"\n⚠️ IMPORTANT: Gardez ce fichier de backup pour pouvoir restaurer si nécessaire !")
    print(f"\n📊 Aperçu des données:")
    print(applications_df.head())
    
    # Stocker le nom du fichier de mapping pour la restauration
    RESTORE_MAPPING_FILE = restore_mapping_filename if 'restore_mapping_filename' in locals() else None
    
else:
    print(f"❌ Erreur lors de la récupération: {response.status_code}")
    print(f"Message: {response.text}")
    applications_df = None
    RESTORE_MAPPING_FILE = None


📥 Récupération des candidatures depuis l'API...
Réponse API: <Response [200]>

✅ Mapping de restauration sauvegardé dans: applications/restore_mapping_20251215_210249.csv
   Ce fichier permet de restaurer toutes les sources originales facilement

✅ 5245 candidatures récupérées
✅ Backup complet sauvegardé dans: applications/all_applications_backup_20251215_210249.csv

⚠️ IMPORTANT: Gardez ce fichier de backup pour pouvoir restaurer si nécessaire !

📊 Aperçu des données:
    id  status recordingUrl notes  tags                  slug     source  \
0   34  denied         None  None  [[]]  zCEueXCowngAtSZ9zJ0s  hellowork   
1   36     new         None  None    []  uQz6CnEvs0EWATAnmbum  hellowork   
2   53  denied         None  None    []  v9UWAGUjEdEx8rG53GxN  hellowork   
3   68     new         None  None    []  x7ffzApszM1IFRWxw4TU  hellowork   
4  223     new         None  None    []  MJEdSStQ1sKIgXhx7Msu  hellowork   

     campaign    applicant           cv                created_at  \


## 📊 Étape 5 : Analyse des sources existantes

Cette cellule analyse toutes les sources différentes présentes dans les candidatures.

**Résultat :** Un tableau montrant :
- Toutes les sources uniques
- Le nombre de candidatures pour chaque source
- Quelles sources seront modifiées


In [8]:
if applications_df is not None and 'source' in applications_df.columns:
    print("📊 Analyse des sources existantes:\n")
    print("=" * 60)
    
    # Compter les sources
    source_counts = applications_df['source'].value_counts().sort_index()
    
    print(f"\nNombre total de sources différentes: {len(source_counts)}")
    print(f"\nRépartition par source:")
    print("-" * 60)
    
    for source, count in source_counts.items():
        status = "✅ CONSERVÉE" if source == SOURCE_TO_KEEP else "🔄 SERA MODIFIÉE"
        print(f"{source:<30} : {count:>6} candidatures  {status}")
    
    # Calculer combien seront modifiées
    total_to_modify = len(applications_df[applications_df['source'] != SOURCE_TO_KEEP])
    total_to_keep = len(applications_df[applications_df['source'] == SOURCE_TO_KEEP])
    
    print("\n" + "=" * 60)
    print(f"\n📈 Résumé:")
    print(f"   Total candidatures: {len(applications_df)}")
    print(f"   À conserver ('{SOURCE_TO_KEEP}'): {total_to_keep}")
    print(f"   À modifier → '{NEW_SOURCE}': {total_to_modify}")
    print("\n" + "=" * 60)
    
    # Sauvegarder l'analyse
    source_analysis = pd.DataFrame({
        'source': source_counts.index,
        'count': source_counts.values,
        'will_be_modified': source_counts.index != SOURCE_TO_KEEP
    })
    source_analysis.to_csv('applications/source_analysis.csv', index=False)
    print("\n✅ Analyse sauvegardée dans: applications/source_analysis.csv")
    
else:
    print("❌ Impossible d'analyser les sources. Vérifiez que les candidatures ont été récupérées.")


📊 Analyse des sources existantes:


Nombre total de sources différentes: 7

Répartition par source:
------------------------------------------------------------
cabine cibli job               :   1046 candidatures  ✅ CONSERVÉE
campaign_share                 :   1105 candidatures  🔄 SERA MODIFIÉE
career_site                    :    154 candidatures  🔄 SERA MODIFIÉE
hellowork                      :   2788 candidatures  🔄 SERA MODIFIÉE
jobijoba                       :     95 candidatures  🔄 SERA MODIFIÉE
recruiter_cvtheque             :      1 candidatures  🔄 SERA MODIFIÉE
recruiter_manual               :      2 candidatures  🔄 SERA MODIFIÉE


📈 Résumé:
   Total candidatures: 5245
   À conserver ('cabine cibli job'): 1046
   À modifier → 'cibli': 4199


✅ Analyse sauvegardée dans: applications/source_analysis.csv


In [11]:
if applications_df is not None:
    # Filtrer les candidatures à modifier
    applications_to_modify = applications_df[applications_df['source'] != SOURCE_TO_KEEP].copy()
    
    print(f"🔄 Modification de {len(applications_to_modify)} candidatures...")
    print(f"   Source actuelle: toutes sauf '{SOURCE_TO_KEEP}'")
    print(f"   Nouvelle source: '{NEW_SOURCE}'")
    print("\n" + "=" * 60)
    
    # Préparer le rapport
    modification_report = []
    success_count = 0
    error_count = 0
    
    # Modifier chaque candidature
    for idx, application in applications_to_modify.iterrows():
        application_id = application.get('id')
        old_source = application.get('source')
        
        if application_id is None:
            print(f"⚠️ Candidature sans ID à l'index {idx}, ignorée")
            error_count += 1
            continue
        
        # Préparer les données de mise à jour
        update_data = {
            "source": NEW_SOURCE
        }
        
        # Endpoint pour mettre à jour une candidature
        update_endpoint = f"{API_URL}/applications/{application_id}"
        
        try:
            # Faire la requête PUT/PATCH
            update_response = requests.put(
                update_endpoint,
                headers={
                    "x-api-key": API_KEY,
                    "Content-Type": "application/json"
                },
                json=update_data
            )

            if update_response.status_code in [200, 204]:
                success_count += 1
                status = "✅"
            else:
                error_count += 1
                status = f"❌ ({update_response.status_code})"
            
            # Enregistrer dans le rapport
            modification_report.append({
                'application_id': application_id,
                'old_source': old_source,
                'new_source': NEW_SOURCE,
                'status_code': update_response.status_code,
                'success': update_response.status_code in [200, 204],
                'error_message': update_response.text if update_response.status_code not in [200, 204] else None
            })
            
            # Afficher le progrès tous les 50 éléments
            if (success_count + error_count) % 50 == 0:
                print(f"   Progression: {success_count + error_count}/{len(applications_to_modify)} ({status})")
                
        except Exception as e:
            error_count += 1
            modification_report.append({
                'application_id': application_id,
                'old_source': old_source,
                'new_source': NEW_SOURCE,
                'status_code': None,
                'success': False,
                'error_message': str(e)
            })
            print(f"❌ Erreur pour candidature {application_id}: {e}")
    
    # Créer le DataFrame du rapport
    report_df = pd.DataFrame(modification_report)
    
    # Sauvegarder le rapport
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    report_filename = f'applications/modification_report_{timestamp}.csv'
    report_df.to_csv(report_filename, index=False)
    
    # Afficher le résumé
    print("\n" + "=" * 60)
    print("📊 RÉSUMÉ DES MODIFICATIONS")
    print("=" * 60)
    print(f"✅ Succès: {success_count}")
    print(f"❌ Erreurs: {error_count}")
    print(f"📝 Total traité: {len(modification_report)}")
    print(f"\n✅ Rapport sauvegardé dans: {report_filename}")
    
    # Afficher les erreurs s'il y en a
    if error_count > 0:
        print("\n⚠️ Erreurs rencontrées:")
        errors_df = report_df[report_df['success'] == False]
        print(errors_df.head(10))
        
else:
    print("❌ Impossible de modifier les candidatures. Vérifiez que les données ont été récupérées.")


🔄 Modification de 4199 candidatures...
   Source actuelle: toutes sauf 'cabine cibli job'
   Nouvelle source: 'cibli'

   Progression: 50/4199 (✅)
   Progression: 100/4199 (✅)
   Progression: 150/4199 (✅)
   Progression: 200/4199 (✅)
   Progression: 250/4199 (✅)
   Progression: 300/4199 (✅)
   Progression: 350/4199 (✅)
   Progression: 400/4199 (✅)
   Progression: 450/4199 (✅)
   Progression: 500/4199 (✅)
   Progression: 550/4199 (✅)
   Progression: 600/4199 (✅)
   Progression: 650/4199 (✅)
   Progression: 700/4199 (✅)
   Progression: 750/4199 (✅)
   Progression: 800/4199 (✅)
   Progression: 850/4199 (✅)
   Progression: 900/4199 (✅)


KeyboardInterrupt: 

In [ ]:
print("🔍 Vérification des modifications...")
print("=" * 60)

# Récupérer les données mises à jour
verification_response = requests.get(
    APPLICATIONS_ENDPOINT,
    headers={"x-api-key": API_KEY}
)

if verification_response.status_code == 200:
    updated_applications = verification_response.json()
    updated_df = pd.DataFrame(updated_applications)
    
    # Sauvegarder les données mises à jour
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    updated_filename = f'applications/all_applications_updated_{timestamp}.csv'
    updated_df.to_csv(updated_filename, index=False)
    
    print(f"✅ {len(updated_df)} candidatures récupérées après modification")
    
    # Analyser les sources après modification
    if 'source' in updated_df.columns:
        updated_source_counts = updated_df['source'].value_counts()
        
        print("\n📊 Sources après modification:")
        print("-" * 60)
        for source, count in updated_source_counts.items():
            print(f"{source:<30} : {count:>6} candidatures")
        
        # Vérifier que les modifications ont bien été appliquées
        expected_sources = {SOURCE_TO_KEEP, NEW_SOURCE}
        actual_sources = set(updated_source_counts.index)
        
        print("\n" + "=" * 60)
        if actual_sources == expected_sources:
            print("✅ Vérification réussie ! Les sources sont correctes.")
        else:
            print("⚠️ Attention: Des sources inattendues ont été trouvées:")
            unexpected = actual_sources - expected_sources
            for source in unexpected:
                print(f"   - {source}: {updated_source_counts[source]} candidatures")
        
        print(f"\n✅ Données mises à jour sauvegardées dans: {updated_filename}")
    
else:
    print(f"❌ Erreur lors de la vérification: {verification_response.status_code}")


---

## ✅ C'est terminé !

### 📁 Fichiers générés :

1. **Backup initial** : `applications/all_applications_backup_YYYYMMDD_HHMMSS.csv`
   - ⚠️ **IMPORTANT :** Gardez ce fichier pour pouvoir restaurer !
2. **Mapping de restauration** : `applications/restore_mapping_YYYYMMDD_HHMMSS.csv`
   - Mapping ID → source originale pour restauration facile
3. **Analyse des sources** : `applications/source_analysis.csv`
4. **Rapport de modification** : `applications/modification_report_YYYYMMDD_HHMMSS.csv`
5. **Données finales** : `applications/all_applications_updated_YYYYMMDD_HHMMSS.csv`

### 📥 Pour télécharger les fichiers :

1. Cliquez sur l'icône **📁 "Fichiers"** dans la barre latérale gauche
2. Faites un **clic droit** sur chaque fichier
3. Sélectionnez **"Télécharger"** (Download)

### 🔄 Pour restaurer les données (si nécessaire) :

Si le frontend ne peut pas afficher la nouvelle source "cibli", utilisez la cellule de restauration ci-dessous.

---

**Merci d'avoir utilisé ce notebook ! 🎉**


---

# 🔄 RESTAURATION DES SOURCES ORIGINALES

**⚠️ UTILISEZ CETTE SECTION UNIQUEMENT SI VOUS VOULEZ RESTAURER LES SOURCES ORIGINALES**

Cette section permet de restaurer toutes les sources à leur valeur d'origine depuis le backup.

**Quand utiliser cette section :**
- Le frontend ne peut pas afficher la nouvelle source "cibli"
- Vous voulez annuler les modifications
- Vous avez besoin de revenir à l'état précédent

**⚠️ ATTENTION :** Cette opération va modifier toutes les candidatures qui ont été changées !


## 🔙 Étape de Restauration 1 : Configuration

**⚠️ IMPORTANT :** Spécifiez le nom du fichier de mapping de restauration créé lors du backup.

Le fichier devrait s'appeler : `applications/restore_mapping_YYYYMMDD_HHMMSS.csv`

**Si vous ne connaissez pas le nom exact :**
1. Regardez dans le panneau de fichiers à gauche
2. Cherchez les fichiers commençant par `restore_mapping_`
3. Utilisez le plus récent (celui créé juste avant les modifications)


In [ ]:
# Configuration pour la restauration
# ⚠️ MODIFIEZ CE NOM DE FICHIER avec le nom exact de votre fichier de mapping
RESTORE_MAPPING_FILE = "applications/restore_mapping_20251215_210249.csv"  # ← MODIFIEZ ICI

# Vérifier que le fichier existe
if os.path.exists(RESTORE_MAPPING_FILE):
    print(f"✅ Fichier de mapping trouvé: {RESTORE_MAPPING_FILE}")
    restore_df = pd.read_csv(RESTORE_MAPPING_FILE)
    print(f"✅ {len(restore_df)} candidatures à restaurer")
    print(f"\n📊 Aperçu du mapping:")
    print(restore_df.head(10))
    print(f"\n📊 Sources à restaurer:")
    source_counts = restore_df['original_source'].value_counts()
    for source, count in source_counts.items():
        print(f"   {source}: {count} candidatures")
else:
    print(f"❌ Fichier non trouvé: {RESTORE_MAPPING_FILE}")
    print(f"\n📁 Fichiers disponibles dans 'applications/':")
    if os.path.exists('applications'):
        files = [f for f in os.listdir('applications') if f.startswith('restore_mapping_')]
        if files:
            print("   Fichiers de mapping trouvés:")
            for f in sorted(files, reverse=True):  # Plus récent en premier
                print(f"   - {f}")
        else:
            print("   Aucun fichier de mapping trouvé")
    restore_df = None


✅ Fichier de mapping trouvé: applications/restore_mapping_20251215_210249.csv
✅ 5245 candidatures à restaurer

📊 Aperçu du mapping:
   application_id original_source
0              34       hellowork
1              36       hellowork
2              53       hellowork
3              68       hellowork
4             223       hellowork
5             241       hellowork
6             242       hellowork
7             257       hellowork
8             259             NaN
9             274       hellowork

📊 Sources à restaurer:
   hellowork: 2788 candidatures
   campaign_share: 1105 candidatures
   cabine cibli job: 1046 candidatures
   career_site: 154 candidatures
   jobijoba: 95 candidatures
   recruiter_manual: 2 candidatures
   recruiter_cvtheque: 1 candidatures


## 🔙 Étape de Restauration 2 : Restauration des sources

**⚠️ ATTENTION : Cette cellule va restaurer toutes les sources originales dans l'API !**

Cette cellule :
- Lit le mapping de restauration
- Restaure chaque candidature à sa source originale
- Génère un rapport de restauration

**Temps estimé :** 1-5 minutes selon le nombre de candidatures


In [16]:
if restore_df is not None:
    print("🔄 Restauration des sources originales...")
    print("=" * 60)
    
    # Préparer le rapport
    restore_report = []
    success_count = 0
    error_count = 0
    
    # Restaurer chaque candidature
    for idx, row in restore_df.iterrows():
        application_id = row['application_id']
        original_source = row['original_source']

        if pd.isna(original_source):
            print(f" Fallback to hellowork")
            original_source = "hellowork"
        if pd.isna(application_id):
            print(f"⚠️ Ligne {idx} ignorée (données manquantes)")
            error_count += 1
            continue
        
        # Préparer les données de restauration
        restore_data = {
            "source": original_source
        }
        
        # Endpoint pour mettre à jour une candidature
        update_endpoint = f"{API_URL}/applications/{application_id}"
        
        try:
            # Faire la requête PATCH
            restore_response = requests.put(
                update_endpoint,
                headers={
                    "x-api-key": API_KEY,
                    "Content-Type": "application/json"
                },
                json=restore_data
            )
            
            if restore_response.status_code in [200, 204]:
                success_count += 1
                status = "✅"
            else:
                error_count += 1
                status = f"❌ ({restore_response.status_code})"
            
            # Enregistrer dans le rapport
            restore_report.append({
                'application_id': application_id,
                'restored_source': original_source,
                'status_code': restore_response.status_code,
                'success': restore_response.status_code in [200, 204],
                'error_message': restore_response.text if restore_response.status_code not in [200, 204] else None
            })
            
            # Afficher le progrès tous les 50 éléments
            if (success_count + error_count) % 50 == 0:
                print(f"   Progression: {success_count + error_count}/{len(restore_df)} ({status})")
                
        except Exception as e:
            error_count += 1
            restore_report.append({
                'application_id': application_id,
                'restored_source': original_source,
                'status_code': None,
                'success': False,
                'error_message': str(e)
            })
            print(f"❌ Erreur pour candidature {application_id}: {e}")
    
    # Créer le DataFrame du rapport
    restore_report_df = pd.DataFrame(restore_report)
    
    # Sauvegarder le rapport
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    restore_report_filename = f'applications/restore_report_{timestamp}.csv'
    restore_report_df.to_csv(restore_report_filename, index=False)
    
    # Afficher le résumé
    print("\n" + "=" * 60)
    print("📊 RÉSUMÉ DE LA RESTAURATION")
    print("=" * 60)
    print(f"✅ Succès: {success_count}")
    print(f"❌ Erreurs: {error_count}")
    print(f"📝 Total traité: {len(restore_report)}")
    print(f"\n✅ Rapport sauvegardé dans: {restore_report_filename}")
    
    # Afficher les erreurs s'il y en a
    if error_count > 0:
        print("\n⚠️ Erreurs rencontrées:")
        errors_df = restore_report_df[restore_report_df['success'] == False]
        print(errors_df.head(10))
    else:
        print("\n🎉 Toutes les sources ont été restaurées avec succès !")
        
else:
    print("❌ Impossible de restaurer. Vérifiez que le fichier de mapping a été chargé.")


🔄 Restauration des sources originales...
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
   Progression: 50/5245 (✅)
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
   Progression: 100/5245 (✅)
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
 Fallback to hellowork
   Progression: 150/5245 (✅)
 Fallback to hellowork
   Progression: 200/5245 (✅)
 Fallback to hel

KeyboardInterrupt: 

## 🔙 Étape de Restauration 3 : Vérification de la restauration

Cette cellule vérifie que les sources ont bien été restaurées.


In [17]:
print("🔍 Vérification de la restauration...")
print("=" * 60)

# Récupérer les données après restauration
verification_response = requests.get(
    APPLICATIONS_ENDPOINT,
    headers={"x-api-key": API_KEY}
)

if verification_response.status_code == 200:
    restored_applications = verification_response.json()
    restored_df = pd.DataFrame(restored_applications)
    
    # Sauvegarder les données restaurées
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    restored_filename = f'applications/all_applications_restored_{timestamp}.csv'
    restored_df.to_csv(restored_filename, index=False)
    
    print(f"✅ {len(restored_df)} candidatures récupérées après restauration")
    
    # Analyser les sources après restauration
    if 'source' in restored_df.columns and restore_df is not None:
        restored_source_counts = restored_df['source'].value_counts()
        original_source_counts = restore_df['original_source'].value_counts()
        
        print("\n📊 Sources après restauration:")
        print("-" * 60)
        for source, count in restored_source_counts.items():
            original_count = original_source_counts.get(source, 0)
            match = "✅" if count == original_count else "⚠️"
            print(f"{source:<30} : {count:>6} candidatures  {match} (original: {original_count})")
        
        # Comparer avec les sources originales
        print("\n" + "=" * 60)
        print("📊 Comparaison avec les sources originales:")
        
        all_sources = set(restored_source_counts.index) | set(original_source_counts.index)
        all_match = True
        
        for source in sorted(all_sources):
            restored_count = restored_source_counts.get(source, 0)
            original_count = original_source_counts.get(source, 0)
            if restored_count == original_count:
                print(f"✅ {source}: {restored_count} candidatures (identique)")
            else:
                print(f"⚠️ {source}: {restored_count} restaurées vs {original_count} originales")
                all_match = False
        
        if all_match:
            print("\n🎉 RESTAURATION RÉUSSIE ! Toutes les sources correspondent aux originales.")
        else:
            print("\n⚠️ Certaines sources ne correspondent pas exactement. Vérifiez le rapport.")
        
        print(f"\n✅ Données restaurées sauvegardées dans: {restored_filename}")
    
else:
    print(f"❌ Erreur lors de la vérification: {verification_response.status_code}")


🔍 Vérification de la restauration...
✅ 5245 candidatures récupérées après restauration

📊 Sources après restauration:
------------------------------------------------------------
hellowork                      :   2837 candidatures  ⚠️ (original: 2788)
campaign_share                 :   1105 candidatures  ✅ (original: 1105)
cabine cibli job               :   1046 candidatures  ✅ (original: 1046)
career_site                    :    154 candidatures  ✅ (original: 154)
jobijoba                       :     95 candidatures  ✅ (original: 95)
recruiter_manual               :      2 candidatures  ✅ (original: 2)
recruiter_cvtheque             :      1 candidatures  ✅ (original: 1)

📊 Comparaison avec les sources originales:
✅ cabine cibli job: 1046 candidatures (identique)
✅ campaign_share: 1105 candidatures (identique)
✅ career_site: 154 candidatures (identique)
⚠️ hellowork: 2837 restaurées vs 2788 originales
✅ jobijoba: 95 candidatures (identique)
✅ recruiter_cvtheque: 1 candidatures (ident

---

## ✅ Restauration terminée !

Les sources ont été restaurées à leur valeur d'origine.

**📁 Fichiers générés :**
- `applications/restore_report_YYYYMMDD_HHMMSS.csv` - Rapport de restauration
- `applications/all_applications_restored_YYYYMMDD_HHMMSS.csv` - Données après restauration

**💡 Astuce :** Comparez les fichiers de backup et de restauration pour vérifier que tout correspond.
